In [1]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [2]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [4]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 16813124 (64.14 MB)
Trainable params: 16813124 (64.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
conv_base.trainable = False

In [7]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='../Alzheimer/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='training'  # Use this subset for training
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='../Alzheimer/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='validation'  # Use this subset for validation
)

Found 5121 files belonging to 4 classes.
Found 1279 files belonging to 4 classes.


In [8]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10


161/161 [==============================] - 531s 3s/step - loss: 1.0040 - accuracy: 0.5542 - val_loss: 0.9477 - val_accuracy: 0.5520
Epoch 2/10
161/161 [==============================] - 574s 4s/step - loss: 0.7920 - accuracy: 0.6372 - val_loss: 0.9988 - val_accuracy: 0.5958
Epoch 3/10
161/161 [==============================] - 428s 3s/step - loss: 0.6812 - accuracy: 0.7051 - val_loss: 0.8719 - val_accuracy: 0.6224
Epoch 4/10
161/161 [==============================] - 388s 2s/step - loss: 0.5888 - accuracy: 0.7536 - val_loss: 1.0056 - val_accuracy: 0.5715
Epoch 5/10
161/161 [==============================] - 421s 3s/step - loss: 0.5239 - accuracy: 0.7782 - val_loss: 0.9482 - val_accuracy: 0.6443
Epoch 6/10
161/161 [==============================] - 361s 2s/step - loss: 0.4264 - accuracy: 0.8397 - val_loss: 1.0093 - val_accuracy: 0.6450
Epoch 7/10
161/161 [==============================] - 351s 2s/step - loss: 0.4030 - accuracy: 0.8385 - val_loss: 0.8928 - val_accuracy: 0.64